### Testing the logic

In [1]:
import mlflow
import pandas as pd
import numpy as np

In [15]:
# The data that will be sent in the request body
player = {
  "gp": 0.0,
  "min": 0.0,
  "pts": 0.0,
  "fgm": 0.0,
  "fga": 0.0,
  "fgpercent": 0.0,
  "treepmade": 0.0,
  "treepa": 0.0,
  "treeppercent": 0.0,
  "ftm": 0.0,
  "fta": 0.0,
  "ftpercent": 0.0,
  "oreb": 0.0,
  "dreb": 0.0,
  "reb": 0.0,
  "ast": 0.0,
  "stl": 0.0,
  "blk": 0.0,
  "tov": 0.0
}

In [3]:
# set tracking uri
mlflow.set_tracking_uri("http://20.224.70.229:5000/")

# set experiment name
mlflow.set_experiment("nba-investment-experiment")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1673524780458, experiment_id='1', last_update_time=1673524780458, lifecycle_stage='active', name='nba-investment-experiment', tags={}>

In [4]:
# defining the model from mlfow registry
logged_model = 'runs:/92ba1717068040aba1137975e81e4b81/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

2023/01/14 19:14:39 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - psutil (current: 5.9.4, required: psutil==5.9.0)
 - typing-extensions (current: 4.4.0, required: typing-extensions==4.2.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [5]:
columns=["GP", "MIN", "PTS", "FGM", "FGA", "FG%", "3P Made", "3PA", "3P%", "FTM", "FTA", "FT%", "OREB", "DREB", "REB", "AST", "STL", "BLK", "TOV"]
input_data = pd.DataFrame(np.array(list(player.values())).reshape(1,19), columns=columns)
input_data

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
loaded_model.predict(input_data)

array([0.19239946], dtype=float32)

### Testing the API

In [2]:
import json
import requests

# The data that will be sent in the request body
player = {
  "gp": 0.0,
  "min": 0.0,
  "pts": 0.0,
  "fgm": 0.0,
  "fga": 0.0,
  "fgpercent": 0.0,
  "treepmade": 0.0,
  "treepa": 0.0,
  "treeppercent": 0.0,
  "ftm": 0.0,
  "fta": 0.0,
  "ftpercent": 0.0,
  "oreb": 0.0,
  "dreb": 0.0,
  "reb": 0.0,
  "ast": 0.0,
  "stl": 0.0,
  "blk": 0.0,
  "tov": 0.0
}

# The url of the endpoint that the request will be sent to
url = "http://localhost:4100/online_predict"

# The headers of the request, in this case it's setting the accept and Content-Type
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

# Send the request
response = requests.post(url, headers=headers, json=player)

# Print the json data from the response
print(response.json())

{'prediction': 0, 'status': 200, 'message': 'Bad investment'}


In [2]:
import requests

url = "http://localhost:4100/batch_predict"

payload={}
files=[
  ('file',('response_1673789264952.csv',open('/Users/abdessamadbaahmed/Downloads/response_1673789264952.csv','rb'),'text/csv'))
]
headers = {}

response = requests.request("POST", url, headers=headers, data=payload, files=files)

print(response.text)

{"file_name":"predictions_e41f97d6-f772-4b76-8fee-bae50a45c619.csv","status":200}


In [15]:
from minio import Minio
import pandas as pd
minio_client = Minio('localhost:9000',
        access_key='abdessamadbaahmed',
        secret_key='baahmedabdessamad', secure=False)

def read_csv_from_minio(client, bucket_name, object_name):
    """
    The function read a csv file from a minio bucket
    
    :param client: the minio client
    :param bucket_name: the name of the bucket
    :param object_name: the name of the object
    
    :return: DataFrame
    """
    try:
        # Get the object data
        data = client.get_object(bucket_name, object_name)
        return pd.read_csv(data)
    except Exception as e:
        print(e)

#response = requests.request("POST", url, headers={}, data={}, files=files).json()
#response
read_csv_from_minio(minio_client, "nba-investment-data", "predictions_1bfe0f43-9488-4d2e-8d9f-d2f3ca051762.csv")

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs,predictions
0,36.0,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,...,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0.0,0.077075
1,35.0,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,...,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0.0,0.378277
2,74.0,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,0.9,...,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0.0,0.032334
3,58.0,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,...,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1.0,0.311863
4,48.0,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,1.3,...,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1.0,0.952707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,80.0,15.8,4.3,1.6,3.6,43.3,0.0,0.2,14.3,1.2,...,79.2,0.4,0.8,1.2,2.5,0.6,0.2,0.8,0.0,0.256241
1336,68.0,12.6,3.9,1.5,4.1,35.8,0.1,0.7,16.7,0.8,...,79.4,0.4,1.1,1.5,2.3,0.8,0.0,1.3,1.0,0.889927
1337,43.0,12.1,5.4,2.2,3.9,55.0,0.0,0.0,0.0,1.0,...,64.3,1.5,2.3,3.8,0.3,0.3,0.4,0.9,0.0,0.769029
1338,52.0,12.0,4.5,1.7,3.8,43.9,0.0,0.2,10.0,1.2,...,62.5,0.2,0.4,0.7,2.2,0.4,0.1,0.8,1.0,0.934679


In [11]:
def list_objects_in_minio_bucket(client, bucket_name):
    """
    The function list all the objects in a minio bucket
    
    :param client: the minio client
    :param bucket_name: the name of the bucket
    
    :return: list of objects
    """
    try:
        # List all objects in the bucket
        objects = client.list_objects(bucket_name)
        return [obj.object_name for obj in objects]
    except Exception as e:
        print(e)

list_objects_in_minio_bucket(minio_client, "nba-investment-data")

['nba_logreg_preprocessed.csv',
 'nba_logreg_processed_test.csv',
 'nba_logreg_processed_train.csv',
 'nba_logreg_raw.csv',
 'predictions_019b8578-17ae-4382-8114-5820140b5726.csv',
 'predictions_08376465-03ac-4195-ae66-e4ee662aa0d2.csv',
 'predictions_1bfe0f43-9488-4d2e-8d9f-d2f3ca051762.csv',
 'predictions_2276ca7b-daa6-432c-ba92-6cb236c68bdf.csv',
 'predictions_26bc8fe5-37ef-4622-83fa-b5bd726e2d6e.csv',
 'predictions_2e548f9b-acce-494c-baf1-9af92f1c6223.csv',
 'predictions_2e813644-0260-4ce6-958b-5a68a0e4a867.csv',
 'predictions_3b53ea25-ce7e-472f-a8fa-9cf2fb02f073.csv',
 'predictions_458c3685-bce5-4b9c-9f5a-1808c0456c90.csv',
 'predictions_542db8ac-626a-44f8-bf64-a9f60f95d94f.csv',
 'predictions_5c67a5e9-cedb-4c6f-96f5-d75b6507db89.csv',
 'predictions_8157720b-47e1-4d0f-bb84-acf2ee88930c.csv',
 'predictions_83f97942-7fe5-44fe-9193-19165f0fca64.csv',
 'predictions_93dea5fb-715d-408a-97bb-6ce9dccbfbf9.csv',
 'predictions_a3c4c10f-079c-438b-9b4f-1abecbae502e.csv',
 'predictions_a44ee79